In [1]:
import pandas as pd
import itertools
from collections import Counter
import json
from dotenv import load_dotenv
import os
import openai
import os
import matplotlib.pyplot as plt
import requests
import azure
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI
import evaluate
import urllib.request
import ssl
import numpy as np
from scipy import stats

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


#### Load configuration file

In [3]:
import os
import yaml
from datetime import datetime
snapshot_date = datetime.now().strftime("%Y-%m-%d")

with open('config.yaml') as f:
    d = yaml.load(f, Loader=yaml.FullLoader)
    
AZURE_SUBSCRIPTION_ID = d['config']['AZURE_SUBSCRIPTION_ID']
AZURE_RESOURCE_GROUP = d['config']['AZURE_RESOURCE_GROUP']
AZURE_WORKSPACE = d['config']['AZURE_WORKSPACE']
AZURE_DATA_NAME = d['config']['AZURE_DATA_NAME']    
DATA_DIR = d['config']['DATA_DIR']
CLOUD_DIR = d['config']['CLOUD_DIR']
HF_MODEL_NAME_OR_PATH = d['config']['HF_MODEL_NAME_OR_PATH']
IS_DEBUG = d['config']['IS_DEBUG']
USE_LOWPRIORITY_VM = d['config']['USE_LOWPRIORITY_VM']


rest_endpoint = d['evaluation']['rest_endpoint']
evaluation_deployment_name = d['evaluation']['deployment_name']

### Configure workspace

In [4]:
import time
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml import command
from azure.ai.ml.entities import Data, Environment, BuildContext
from azure.ai.ml.entities import Model
from azure.ai.ml import Input
from azure.ai.ml import Output
from azure.ai.ml.constants import AssetTypes
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    ml_client = MLClient(credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE)

Found the config file in: /config.json


### Format csv to json

In [ ]:
import pandas as pd
df = pd.read_csv("../analysis_and_tools_only.csv")

In [6]:
#preprocess data

input_string = []

# Iterate through the rows of the dataframe
for index, row in df.iterrows():
    # 'content' is the column with the user questions
    user_content = row['content']
    # 'answer content' is the column with the ground truth answers
    assistant_content = row['answer_content']

    # Append only user content to the input_data
    input_string.append({"role": "user", "content": user_content})

# Define the parameters for the model
params = {
    "temperature": 0.1,
    "max_new_tokens": 4096, #output
    "do_sample": True,
    "return_full_text": False
}

# Combine input data and parameters into the final test data structure
test_data = {
    "input_data": input_string,
    "parameters": params
}

# Save the JSON output to a file
with open('test_data.json', 'w') as json_file:
    json.dump(test_data, json_file, indent=4)

print("JSON file created successfully.")

JSON file created successfully.


### Evaluate model 

#### Evaulation real data

In [7]:
# Define the path for saving the real request file
test_src_dir = "./phi3-inference-test"
os.makedirs(test_src_dir, exist_ok=True)
real_data_path = os.path.join(test_src_dir, "test_data.json")

# Save the real request data to a JSON file
with open(real_data_path, "w") as f:
    json.dump(test_data, f, indent=4)


In [8]:
# Define paths and parameters
timeout = 1000
test_src_dir = "./phi3-inference-test"
response_src_dir = "./phi3-inference-responses"
batch_size = 1

# Ensure directories exist
os.makedirs(test_src_dir, exist_ok=True)
os.makedirs(response_src_dir, exist_ok=True)

def batch_data(data, batch_size):
    """Split data into batches of given size."""
    return [data[i:i + batch_size] for i in range(0, len(data), batch_size)]


# Create batches
batches = batch_data(test_data["input_data"], batch_size)

In [9]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.


In [ ]:
def submit_tasks(query, id):
    data = {
    "input_data": {
        "input_string": batch,
        "parameters": {
        "temperature": 0.1,
        "max_new_tokens": 4096
        }
    }
    }
    batch_file_path = os.path.join(test_src_dir, f"batch_{id}.json")
    # Save the batch request data to a JSON file
    with open(batch_file_path, "w") as f:
        json.dump(data, f, indent=4)

    body = str.encode(json.dumps(data))
    ##print(f"Batch request data saved to: {batch_file_path}")


    url = ''
    api_key = ''
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")


    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        result_parsed = json.loads(result)  # Convert string to dictionary
        output = result_parsed['output']
        
        response_file_path = os.path.join(response_src_dir, f"response_{id}.json")
        
        # Save both input and response data
        with open(response_file_path, "w") as f:
            json.dump({
                "input_data": batch,
                "response_data": output  # Ensure result is JSON serializable
            }, f, indent=4)
        
        ##print(f"Result for batch {i} saved to: {response_file_path}")
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


In [93]:
## loop through batches
for i, batch in enumerate(batches):
    submit_tasks(batch,i)

Batch request data saved to: ./phi3-inference-test/batch_0.json
Result for batch 0 saved to: ./phi3-inference-responses/response_0.json
Batch request data saved to: ./phi3-inference-test/batch_1.json
Result for batch 1 saved to: ./phi3-inference-responses/response_1.json
Batch request data saved to: ./phi3-inference-test/batch_2.json
Result for batch 2 saved to: ./phi3-inference-responses/response_2.json
Batch request data saved to: ./phi3-inference-test/batch_3.json
Result for batch 3 saved to: ./phi3-inference-responses/response_3.json
Batch request data saved to: ./phi3-inference-test/batch_4.json
Result for batch 4 saved to: ./phi3-inference-responses/response_4.json
Batch request data saved to: ./phi3-inference-test/batch_5.json
Result for batch 5 saved to: ./phi3-inference-responses/response_5.json
Batch request data saved to: ./phi3-inference-test/batch_6.json
Result for batch 6 saved to: ./phi3-inference-responses/response_6.json
Batch request data saved to: ./phi3-inference-te

In [99]:
import glob
import pandas as pd

file_pattern = "./phi3-inference-responses/*.json"

questions = []
answers = []

for file_name in glob.glob(file_pattern): #glob lists all the text files in the current working dir.
    with open(file_name, 'r') as file:
        data = json.load(file)
    

    if "input_data" in data: #retrieves list of dictionaries
        for item in data["input_data"]: #iterates over each dictionary in the list
            if "content" in item:
                questions.append(item["content"])

    if "response_data" in data:
        response = data["response_data"]
        answers.append(response)

final_df = pd.DataFrame({"Question": questions, "Answer": answers})

# Save DataFrame to a CSV file (optional)
final_df.to_csv('combined_questions_answers.csv', index=False)

final_df.head()

,Question,Answer
0,Hi Everyone. I was trying to add help section ...,To prevent the help message from being printe...
1,I am currently using BWA-MEM to map metagenomi...,"BWA-MEM, like many other alignment tools, doe..."
2,Hi all:I recently got quite confused with two ...,"The SAM flags ""supplementary alignment"" and ""..."
3,"Hello, I am pretty new to bioinformatics and t...",Filtering uniquely mapped reads in bioinforma...
4,Samtools can be used to select reads above cer...,You can streamline the process by using `samt...


#### ROUGE

########note: rouge captures (number of n-grams in prediction summary (fine-tuned phi-3 model) that match the reference summary (ground-truth)) / number of n-grams in reference summary

In [101]:
predictions = final_df["Answer"].to_list()
references = df["answer_content"].to_list()

rouge = evaluate.load('rouge') #https://huggingface.co/spaces/evaluate-metric/rouge


results = rouge.compute(predictions=predictions,
                         references=references,
                        use_aggregator=True)

print("ROUGE-1:", round(results["rouge1"], 3))
print("ROUGE-2:", round(results["rouge2"], 3))
print("ROUGE-L:", round(results["rougeL"], 3))
print("ROUGE-Lsum:", round(results["rougeLsum"], 3))

ROUGE-1: 0.127
ROUGE-2: 0.016
ROUGE-L: 0.073
ROUGE-Lsum: 0.092


### Loop 100 times for generating confidence intervals

In [11]:
import glob
import pandas as pd

rouge_df = pd.DataFrame(columns=["ROUGE-1", "ROUGE-2", "ROUGE-L", "ROUGE-Lsum"])

for j in range(100):

    print(f"Iteration {j}")

    ## loop through batches
    for i, batch in enumerate(batches):
        submit_tasks(batch,i)

    file_pattern = "./phi3-inference-responses/*.json"

    questions = []
    answers = []

    for file_name in glob.glob(file_pattern): #glob lists all the text files in the current working dir.
        with open(file_name, 'r') as file:
            data = json.load(file)
        

        if "input_data" in data: #retrieves list of dictionaries
            for item in data["input_data"]: #iterates over each dictionary in the list
                if "content" in item:
                    questions.append(item["content"])

        if "response_data" in data:
            response = data["response_data"]
            answers.append(response)

    final_df = pd.DataFrame({"Question": questions, "Answer": answers})

    # Save DataFrame to a CSV file (optional)
    final_df.to_csv('combined_questions_answers.csv', index=False)

    final_df.head()

    predictions = final_df["Answer"].to_list()
    references = df["answer_content"].to_list()

    rouge = evaluate.load('rouge') #https://huggingface.co/spaces/evaluate-metric/rouge


    results = rouge.compute(predictions=predictions,
                            references=references,
                            use_aggregator=True)
    rouge_df.loc[j] = [
        round(results["rouge1"], 3),
        round(results["rouge2"], 3),
        round(results["rougeL"], 3),
        round(results["rougeLsum"], 3)
    ]

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [2]:
# Calculate Confidence Interval
def confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard Error of Mean
    interval = sem * stats.t.ppf((1 + confidence) / 2., len(data) - 1)
    return mean, interval

In [ ]:
rouge_df.to_csv('./rouge_results.csv', index=False)



In [3]:
rouge_df = pd.read_csv('./rouge_results.csv')

In [4]:
rouge1_mean, rouge1_ci = confidence_interval(rouge_df['ROUGE-1'])
rouge2_mean, rouge2_ci = confidence_interval(rouge_df['ROUGE-2'])
rougeL_mean, rougeL_ci = confidence_interval(rouge_df['ROUGE-L'])
rougeLsum_mean, rougeLsum_ci = confidence_interval(rouge_df['ROUGE-Lsum'])

print(f"ROUGE-1: {rouge1_mean:.3f} ± {rouge1_ci:.3f}")
print(f"ROUGE-2: {rouge2_mean:.3f} ± {rouge2_ci:.3f}")
print(f"ROUGE-L: {rougeL_mean:.3f} ± {rougeL_ci:.3f}")
print(f"ROUGE-Lsum: {rougeLsum_mean:.3f} ± {rougeLsum_ci:.3f}")

ROUGE-1: 0.129 ± 0.000
ROUGE-2: 0.015 ± 0.000
ROUGE-L: 0.074 ± 0.000
ROUGE-Lsum: 0.092 ± 0.000
